# Set up + log in to Hugging face

In [33]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/opt/conda/bin/hf", line 7, in <module>
    sys.exit(main())
             ~~~~^^
  File "/opt/conda/lib/python3.13/site-packages/huggingface_hub/cli/hf.py", line 59, in main
    service.run()
    ~~~~~~~~~~~^^
  File "/opt/conda/lib

In [34]:
# This is Kas's token, i think this might work when other use it
    # NLP_test Hugging face token 
    # then say "n" for "Add token as git credential? (Y/n)"

In [35]:
import json
import datasets
import pandas as pd

ModuleNotFoundError: No module named 'datasets'

In [ ]:
pd.set_option('display.max_colwidth', None)

## Load job posting df from Hugging Face

In [ ]:
from datasets import load_dataset

ds = load_dataset("lof223/job_description")
df = ds["train"].to_pandas()


In [ ]:
ds

In [ ]:
df.info()

In [ ]:
df.head(2)

# Glimpes through the data

In [ ]:
# check location
df['location'].unique()

## Filter only tech jobs in the US

In [ ]:
df_new = df[['Job Title', 'Company', 'Job Posting Date', 'Job Description', 'location', 'Country']]

In [ ]:
# all text lowercase, so filtering is easier
df_new['Job Title'] = df_new['Job Title'].str.lower()
df_new['Country'] = df_new['Country'].str.lower()
df_new['location'] = df_new['location'].str.lower()

# define a list of tech job keywords
tech_jobs = ['software developer', 'software engineer', 'analyst',
             'data analyst', 'business analyst', 'revenue operations analyst',
             'data scientist', 'developer']


In [ ]:
# filter rows
df_filtered = df_new[
    ((df_new['Country'].str.contains('usa')) |
     (df_new['Country'].str.contains('united states'))) &
    (df_new['Job Title'].str.contains('|'.join(tech_jobs)))
]

In [ ]:
df_filtered.info()

In [ ]:
df_filtered.head(2)

In [ ]:
df_filtered['location'].unique()

### [Notes] USA only have 1078 tech jobs. Even though we not filtering tech jobs, we only have Washington D.C. job posting.

# Source 1 - Hugging Face Job posting data

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(3)

In [ ]:
df_all_jobs = df[['Job Id','Experience','Salary Range','location', 'Country', 'Job Posting Date','Job Title', 'Company', 'Job Description','Benefits','skills','Responsibilities' ]]

In [ ]:
df_all_jobs.info()

## Re-order + add columns to prepare for combination

In [ ]:
df_all_jobs["Company Link"] = None
df_all_jobs["Job Link"] = None

### re-organize + add column to prep for merging with Linkedin Data

In [ ]:
new_order = [
    "Job Id",
    "Job Title",
    "Company",
    "Job Description",
    "Company Link",
    "Job Link",
    "location",
    "Country",
    "Salary Range",
    "Experience",
    "Benefits",
    "skills",
    "Responsibilities"

]

df_all_jobs = df_all_jobs[new_order]

## Save to csv

In [ ]:
# df_all_jobs.to_csv('all_jobs.csv', index=False)

# Source 2 -  Linkedin web scraping data

In [ ]:
linkedin_job = pd.read_json('combined_data.json')

In [ ]:
linkedin_job.info()

In [ ]:
linkedin_job.head(2)

## Clean and extract ID from Linkedin url

In [ ]:
linkedin_job_clean = linkedin_job.copy()

# Extract Job ID
linkedin_job_clean["Job ID"] = linkedin_job_clean["Job Link"].str.extract(r'view/(\d+)/')

# job ID column
cols = linkedin_job_clean.columns.tolist()
cols.insert(0, cols.pop(cols.index("Job ID")))
linkedin_job_clean = linkedin_job_clean[cols]


In [ ]:
linkedin_job_clean = linkedin_job_clean.rename(columns={"Job ID": "Job Id", "Description": "Job Description"})

## Drop some columns from linkedin_job_clean

In [ ]:
# Drop specific columns from the clean dataframe
linkedin_job_clean = linkedin_job_clean.drop(
    ["Date", "Date Text", "Insights", "Description Length"],
    axis=1
)


In [ ]:
# re-order
new_order = [
    "Job Id",
    "Job Title",
    "Company",
    "Job Description",
    "Company Link",
    "Job Link"
]

linkedin_job_clean = linkedin_job_clean[new_order]

In [ ]:
linkedin_job_clean["Job Id"] = linkedin_job_clean["Job Id"].astype("Int64")

In [ ]:
df_all_jobs.info()

In [ ]:
linkedin_job_clean.info()

# Final combined the data set

In [ ]:
missing_cols = [
    "location",
    "Country",
    "Salary Range",
    "Experience",
    "Benefits",
    "skills",
    "Responsibilities"
]

for col in missing_cols:
    linkedin_job_clean[col] = None

In [ ]:
desired_order = df_all_jobs.columns.tolist()

linkedin_job_clean = linkedin_job_clean[desired_order]

df_combined = pd.concat([df_all_jobs, linkedin_job_clean], ignore_index=True)

In [ ]:
df_combined.info()

In [ ]:
df_combined.head()

## Save to csv

In [ ]:
 # df_combined.to_csv('Jobs_data.csv', index=False)

In [31]:
# check last 3 rows

In [32]:
display(df_combined.tail(3))

NameError: name 'df_combined' is not defined